## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP2 - Preparación de los datos

---

En esta libreta se realizan las transforamciones sobre los datasets del escenario en función 
de los resultados de la verificación de calidad de datos. 

---

In [90]:
#Se importan las librerias a utilizar

import pandas as pd

----

##### Lectura de los datasets

In [91]:
df_creditos = pd.read_csv("../../data/processed/datos_creditos_mc.csv", sep=";")
display(df_creditos.head(1))

df_tarjetas = pd.read_csv("../../data/processed/datos_tarjetas_mc.csv", sep=";")
display(df_tarjetas.head(1))

df_integrado = pd.read_csv("../../data/processed/datos_integrados.csv", sep=";")
display(df_integrado.head(1))

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,objetivo_credito_ok,falta_pago_ok
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,0,N,0.0,True,True,True


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,708082083.0,36.0,CASADO,ACTIVO,15149.0,F,3544.0,SECUNDARIO_COMPLETO,Blue,111.0,3.0


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,regla_pct_ingresos,ingreso_regla2
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,...,ACTIVO,15149.0,F,3544.0,SECUNDARIO_COMPLETO,Blue,111.0,3.0,ok,Aprobado


---
#### Aplicación de transformaciones

**Operaciones a realizar**

1. Selección de columnas
2. Filtrado de filas
3. Construcción de atributos
4. Integración de datasets
5. Formateo definitivo


----

Selección de datos

Al realizar un análisis exploratorio, se pudo rescatar ciertas cosas como:
- Se tiene la hipótesis que "nivel_educativo" no interviene en el número de operaciones realizadas.
- El nivel de la tarjeta no parece tener relación con "nivel_credito_tc" ni con "nivel_educativo".
- Se toma en consideración el análisis inicial (Descripción de los datos) para identificar que en la columna "nivel_tarjeta" tiene más del 90% de los datos el valor categórico "Blue", lo que indica que es una tarjeta común. Es así que se considera que la mejor manera de seccionar es a través de el límite crediticio (limite_credito_tc).
- Por los motivos anteriores se decide eliminar las columnas "nivel_educativo" y "nivel_tarjeta".

In [92]:
#El nivel de tarjeta con más usuarios es "Blue"
grouped = df_tarjetas.groupby('nivel_tarjeta')['nivel_educativo'].value_counts()

print(grouped)


nivel_tarjeta  nivel_educativo         
Blue           UNIVERSITARIO_COMPLETO      2899
               UNIVERSITARIO_INCOMPLETO    2331
               SECUNDARIO_COMPLETO         1888
               DESCONOCIDO                 1420
               POSGRADO_INCOMPLETO          476
               POSGRADO_COMPLETO            422
Gold           UNIVERSITARIO_COMPLETO        36
               SECUNDARIO_COMPLETO           26
               UNIVERSITARIO_INCOMPLETO      26
               DESCONOCIDO                   18
               POSGRADO_INCOMPLETO            5
               POSGRADO_COMPLETO              5
Platinum       UNIVERSITARIO_COMPLETO         8
               POSGRADO_INCOMPLETO            3
               UNIVERSITARIO_INCOMPLETO       3
               DESCONOCIDO                    3
               POSGRADO_COMPLETO              2
               SECUNDARIO_COMPLETO            1
Silver         UNIVERSITARIO_COMPLETO       185
               UNIVERSITARIO_INCOMPLETO     140


In [93]:
# Se establece qué columnas se eliminan


col_eliminar_creditos = ['completitud_fila', 'situacion_vivienda_ok', 'objetivo_credito_ok', 'falta_pago_ok'] 

#min 1:24:00 mirar cual borramos de tarjes de credito, hacer el mismo proceso de credito con tarjetas desde la dimension completitud
# ver archivo de aca #Verificacion-calidad.ipynb

col_eliminar_tarjetas = ['nivel_tarjeta', 'nivel_educativo']

#aqui eliminamos la columna creada anteriomente en integrado
col_eliminar_integrado = ['regla_pct_ingresos', 'ingreso_regla2']


# Se ejecuta la operación
df_creditos.drop(col_eliminar_creditos, inplace=True, axis=1)
df_tarjetas.drop(col_eliminar_tarjetas, inplace=True, axis=1)
df_integrado.drop(col_eliminar_integrado, inplace=True, axis=1)

In [94]:
print("Vista del dataset de datos de créditos:")
display(df_creditos.head(1))

print("Vista del dataset de datos de tarjetas:")
display(df_tarjetas.head(1))

print("Vista del dataset de datos integrados:")
display(df_integrado.head(1))

Vista del dataset de datos de créditos:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,0,N


Vista del dataset de datos de tarjetas:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,operaciones_ult_12m,personas_a_cargo
0,708082083.0,36.0,CASADO,ACTIVO,15149.0,F,3544.0,111.0,3.0


Vista del dataset de datos integrados:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,...,36.0,CASADO,ACTIVO,15149.0,F,3544.0,SECUNDARIO_COMPLETO,Blue,111.0,3.0


Limpieza de los datos (Filtrado a nivel de filas)

In [95]:
df_creditos[df_creditos['antiguedad_empleado'] > 50]

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago
2544,713061558.0,22,35000,3,123.0,ALQUILER,59000,PERSONAL,0.59,16.02,1,Y
4138,715999608.0,21,20000,4,123.0,HIPOTECA,192000,INVERSIONES,0.10,6.54,0,N


In [96]:
#Se puede definir una función para aplicar los cálculos
def regla_pct_ingresos_credito(row):
    pct_ingreso = row.pct_ingreso
    ingresos = row.ingresos
    
    if pct_ingreso > 0.5 and ingresos <= 20000:
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'


# Se aplica la función para todos los elementos del dataset
regla_pct_ingresos = df_creditos.apply(lambda row: regla_pct_ingresos_credito(row), axis=1).rename("regla_pct_ingresos")

# Se unen los resultados al dataset inicial
df_creditos = pd.concat([df_creditos, regla_pct_ingresos], axis=1)
df_creditos.head(5)

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,regla_pct_ingresos
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,0,N,ok
1,708083283.0,24,1500,2,0.0,ALQUILER,30996,MEJORAS_HOGAR,0.05,15.99,1,Y,ok
2,708084558.0,23,10000,2,7.0,OTROS,40104,EDUCACIÓN,0.25,12.72,0,Y,ok
3,708085458.0,25,6000,4,2.0,ALQUILER,23198,INVERSIONES,0.26,8.00,0,N,ok
4,708086958.0,26,10000,2,0.0,HIPOTECA,50000,EDUCACIÓN,0.20,7.74,0,N,ok


In [97]:
def verificar_credito(row):
    duracion_credito = row['duracion_credito']
    ingresos = row['ingresos']
    importe_solicitado = row['importe_solicitado']
    situacion_vivienda = row['situacion_vivienda']

    duracion_minima = df_creditos['duracion_credito'].min()

    if duracion_credito == duracion_minima:
        porcentaje_ingresos = (importe_solicitado/ingresos) * 100

        if situacion_vivienda == 'PROPIA':
            return "Aprobado"
        elif porcentaje_ingresos <= 60:
            return "Aprobado"
        else:
            return "Rechazado"
    else:
        return "Aprobado"

#PRUEBA_REGLA_2 = df_creditos.copy()

# Aplicar la función a cada fila del DataFrame
df_creditos['ingreso_regla2'] = df_creditos.apply(verificar_credito, axis=1)

# Se visualizan los datos
print("Se visualizan las tuplas que no cumplen con la regla:\n")
display(df_creditos[df_creditos.ingreso_regla2 == 'Rechazado'].head())

Se visualizan las tuplas que no cumplen con la regla:



,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,regla_pct_ingresos,ingreso_regla2
466,709040508.0,21,15000,2,0.0,HIPOTECA,19500,EDUCACIÓN,0.77,9.64,1,N,err,Rechazado
541,709186983.0,23,20000,2,1.0,ALQUILER,32900,EDUCACIÓN,0.61,16.00,1,N,ok,Rechazado
1646,711399408.0,24,29100,2,8.0,HIPOTECA,46000,PERSONAL,0.63,12.99,1,Y,ok,Rechazado
2573,713115483.0,22,7000,2,0.0,ALQUILER,10000,MEJORAS_HOGAR,0.70,12.21,1,N,err,Rechazado
3041,713962233.0,22,30000,2,1.0,ALQUILER,48000,EDUCACIÓN,0.63,18.39,1,N,ok,Rechazado


In [98]:
# Se filtran las filas con algún error detectado
print(f"Filas antes del filtro: {df_creditos.shape[0]}")

temp = df_creditos[df_creditos['edad'] < 90]
temp_c = temp #Aqui antes estaba: [temp['antiguedad_empleado'] <= 50] #esto se removio en la clase #6 de la actividad practica min 20 al 22
temp_c2 = temp_c[temp_c['regla_pct_ingresos'] == 'ok'] 
# temp_c3 = temp_c2[temp_c2['ingreso_regla2'] == 'Aprobado'] #esto se removio en la clase #6 de la actividad practica min 20 al 22
print(f"Filas despues del filtro: {temp_c2.shape[0]}")


# Otro filtro posible:
# df_creditos['antiguedad_empleado'] > 50

# También por las reglas del negocio
# df[df.regla_pct_ingresos == 'err']


Filas antes del filtro: 10127
Filas despues del filtro: 10108


In [99]:
promedio = temp_c2['tasa_interes'].mean()
temp_c2['tasa_interes'].fillna(promedio, inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_16000\2029723316.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_c2['tasa_interes'].fillna(promedio, inplace=True)


Integración de datos

In [100]:
df_integrado = pd.merge(temp_c2, df_tarjetas, on='id_cliente', how='inner')
coincidencias = df_integrado.shape[0]

print(f"Filas del dataset integrados con los filtros previamente realizados: {coincidencias}")

Filas del dataset integrados con los filtros previamente realizados: 10108


In [101]:
print(f"Cantidad de columnas del dataset integrado: {df_integrado.shape[1]}")

Cantidad de columnas del dataset integrado: 22


In [102]:
display(df_integrado.head(5))

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,regla_pct_ingresos,ingreso_regla2,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,operaciones_ult_12m,personas_a_cargo
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,...,ok,Aprobado,36.0,CASADO,ACTIVO,15149.0,F,3544.0,111.0,3.0
1,708083283.0,24,1500,2,0.0,ALQUILER,30996,MEJORAS_HOGAR,0.05,15.99,...,ok,Aprobado,45.0,SOLTERO,PASIVO,992.0,M,3421.0,21.0,0.0
2,708084558.0,23,10000,2,7.0,OTROS,40104,EDUCACIÓN,0.25,12.72,...,ok,Aprobado,38.0,DIVORCIADO,PASIVO,1447.0,M,8258.0,23.0,3.0
3,708085458.0,25,6000,4,2.0,ALQUILER,23198,INVERSIONES,0.26,8.00,...,ok,Aprobado,24.0,SOLTERO,ACTIVO,3940.0,F,1438.3,82.0,2.0
4,708086958.0,26,10000,2,0.0,HIPOTECA,50000,EDUCACIÓN,0.20,7.74,...,ok,Aprobado,41.0,CASADO,ACTIVO,4369.0,F,3128.0,59.0,2.0


Transformación de atributos

In [103]:
# Columna: estado_civil
cambios_estado_civil = {
    'CASADO' : 'C',
    'SOLTERO' : 'S',
    'DESCONOCIDO' : 'N',
    'DIVORCIADO' : 'D',
}

estado_civil_N = df_integrado.loc[:, ('estado_civil')].map(cambios_estado_civil).rename('estado_civil_N')
df_integrado = pd.concat([estado_civil_N, df_integrado], axis=1)
df_integrado.head()

# Sobre este resultado será necesario eliminar las columnas auxiliares

,estado_civil_N,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,...,regla_pct_ingresos,ingreso_regla2,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,operaciones_ult_12m,personas_a_cargo
0,C,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,...,ok,Aprobado,36.0,CASADO,ACTIVO,15149.0,F,3544.0,111.0,3.0
1,S,708083283.0,24,1500,2,0.0,ALQUILER,30996,MEJORAS_HOGAR,0.05,...,ok,Aprobado,45.0,SOLTERO,PASIVO,992.0,M,3421.0,21.0,0.0
2,D,708084558.0,23,10000,2,7.0,OTROS,40104,EDUCACIÓN,0.25,...,ok,Aprobado,38.0,DIVORCIADO,PASIVO,1447.0,M,8258.0,23.0,3.0
3,S,708085458.0,25,6000,4,2.0,ALQUILER,23198,INVERSIONES,0.26,...,ok,Aprobado,24.0,SOLTERO,ACTIVO,3940.0,F,1438.3,82.0,2.0
4,C,708086958.0,26,10000,2,0.0,HIPOTECA,50000,EDUCACIÓN,0.20,...,ok,Aprobado,41.0,CASADO,ACTIVO,4369.0,F,3128.0,59.0,2.0


In [104]:
# Columna: estado_credito
cambios_estado_credito = {
    0 : 'Cancelado',
    1 : 'Activo',

}

estado_credito_N = df_integrado.loc[:, ('estado_credito')].map(cambios_estado_credito).rename('estado_credito_N')
df_integrado = pd.concat([estado_civil_N, estado_credito_N, df_integrado], axis=1)
df_integrado.head()

# Sobre este resultado será necesario eliminar las columnas auxiliares

,estado_civil_N,estado_credito_N,estado_civil_N,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,...,regla_pct_ingresos,ingreso_regla2,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,operaciones_ult_12m,personas_a_cargo
0,C,Cancelado,C,708082083.0,24,11000,3,5.0,HIPOTECA,64800,...,ok,Aprobado,36.0,CASADO,ACTIVO,15149.0,F,3544.0,111.0,3.0
1,S,Activo,S,708083283.0,24,1500,2,0.0,ALQUILER,30996,...,ok,Aprobado,45.0,SOLTERO,PASIVO,992.0,M,3421.0,21.0,0.0
2,D,Cancelado,D,708084558.0,23,10000,2,7.0,OTROS,40104,...,ok,Aprobado,38.0,DIVORCIADO,PASIVO,1447.0,M,8258.0,23.0,3.0
3,S,Cancelado,S,708085458.0,25,6000,4,2.0,ALQUILER,23198,...,ok,Aprobado,24.0,SOLTERO,ACTIVO,3940.0,F,1438.3,82.0,2.0
4,C,Cancelado,C,708086958.0,26,10000,2,0.0,HIPOTECA,50000,...,ok,Aprobado,41.0,CASADO,ACTIVO,4369.0,F,3128.0,59.0,2.0


Atributos numéricos que se discretizan

In [105]:
df_integrado['operaciones_ult_12m'].describe()

count    10108.000000
mean        64.892659
std         23.465252
min         10.000000
25%         45.000000
50%         67.000000
75%         81.000000
max        139.000000
Name: operaciones_ult_12m, dtype: float64

Ahora veremos uno a uno los atributos integrados

In [106]:
# Antiguedad del empleado

etiquetas_a_e = ['menor_5', '5_a_10', 'mayor_10']
rangos_a_e = [0, 4, 10, 50]
valor_para_nan = 'NA'
antiguedad_empleados_N = pd.cut(df_integrado['antiguedad_empleado'], 
                                bins=rangos_a_e, 
                                labels=etiquetas_a_e,
                                right=False).cat.add_categories(valor_para_nan).fillna(valor_para_nan)

antiguedad_empleados_N.value_counts()

antiguedad_empleado
menor_5     4802
5_a_10      4704
NA           338
mayor_10     264
Name: count, dtype: int64

In [107]:
# edad

etiquetas_e = ['menor_25', '25_a_30']
rangos_e = [0, 24, 50]
edad_N = pd.cut(df_integrado['edad'], 
                                bins=rangos_e, 
                                labels=etiquetas_e)

edad_N.value_counts()

edad
menor_25    7137
25_a_30     2971
Name: count, dtype: int64

In [108]:
# pct_ingreso

etiquetas_p_i = ['hasta_20', '20_a_40', '40_a_60', 'mayor_60']
rangos_p_i = [0, 0.19, 0.39, 0.60, 0.99]
pct_ingreso_N = pd.cut(df_integrado['pct_ingreso'], 
                                bins=rangos_p_i, 
                                labels=etiquetas_p_i)

pct_ingreso_N.value_counts()

pct_ingreso
hasta_20    6445
20_a_40     3191
40_a_60      455
mayor_60      17
Name: count, dtype: int64

In [109]:
# ingresos

etiquetas_i = ['hasta_20k', '20k_a_50k', '50k_a_100k', 'mayor_100k']
rangos_i = [0, 19999, 49999, 99999, 999999]
ingresos_N = pd.cut(df_integrado['ingresos'], 
                                bins=rangos_i, 
                                labels=etiquetas_i)

ingresos_N.value_counts()

ingresos
20k_a_50k     5184
50k_a_100k    3993
hasta_20k      469
mayor_100k     462
Name: count, dtype: int64

In [110]:
# tasa_interes

etiquetas_t_i = ['hasta_7p', '7p_a_15p', '15p_a_20p', 'mayor_20p']
rangos_t_i = [0, 6.99, 14.99, 19.99, 100]
tasa_interes_N = pd.cut(df_integrado['tasa_interes'], 
                                bins=rangos_t_i, 
                                labels=etiquetas_t_i)

tasa_interes_N.value_counts()

tasa_interes
7p_a_15p     7930
hasta_7p     1142
15p_a_20p    1018
mayor_20p      18
Name: count, dtype: int64

In [111]:
# antiguedad_cliente

etiquetas_a_c = ['menor_2y', '2y_a_4y', 'mayor_4y']
rangos_a_c = [0, 24, 48, 100]
antiguedad_cliente_N = pd.cut(df_integrado['antiguedad_cliente'], 
                                bins=rangos_a_c, 
                                labels=etiquetas_a_c)

antiguedad_cliente_N.value_counts()

antiguedad_cliente
2y_a_4y     8608
menor_2y     847
mayor_4y     653
Name: count, dtype: int64

In [112]:
# limite_credito_tc

etiquetas_l_tc = ['menor_3k', '3k_a_5k', '5k_a_10k', 'mayor_10k']
rangos_l_tc = [0, 2999, 4999, 9999, 100000]
limite_credito_tc_N = pd.cut(df_integrado['limite_credito_tc'], 
                                bins=rangos_l_tc, 
                                labels=etiquetas_l_tc)

limite_credito_tc_N.value_counts()

limite_credito_tc
menor_3k     3432
mayor_10k    2749
5k_a_10k     2011
3k_a_5k      1916
Name: count, dtype: int64

In [113]:
# gastos_ult_12m

etiquetas_g_u12 = ['menor_1k', '2k_a_4k', '4k_a_6k', '6k_a_8k', '8k_a_10k', 'mayor_10k']
rangos_g_u12 = [0, 999, 3999, 5999, 7999, 9999, 100000]
gastos_ult_12m_N = pd.cut(df_integrado['gastos_ult_12m'], 
                                bins=rangos_g_u12, 
                                labels=etiquetas_g_u12)

gastos_ult_12m_N.value_counts()

gastos_ult_12m
2k_a_4k      5112
4k_a_6k      3387
mayor_10k     757
6k_a_8k       361
8k_a_10k      331
menor_1k      160
Name: count, dtype: int64

In [114]:
# operaciones_ult_12m

etiquetas_o_u12 = ['menor_15', '15_a_30', '30_a_50', '50_a_75', '75_a_100', 'mayor_100']
rangos_o_u12 = [0, 14, 29, 49, 74, 99, 1000]
operaciones_ult_12m_N = pd.cut(df_integrado['operaciones_ult_12m'], 
                                bins=rangos_o_u12, 
                                labels=etiquetas_o_u12)

operaciones_ult_12m_N.value_counts()

operaciones_ult_12m
50_a_75      3393
75_a_100     3004
30_a_50      2413
mayor_100     687
15_a_30       587
menor_15       24
Name: count, dtype: int64

Exportación de metadatos